In [1]:
from datasets import load_dataset
import pandas as pd

# \t is the tab character in Python
dataset = load_dataset("csv", data_files='compound_properties.csv')
dataset=dataset['train']


# def addsplit(example):
#     return {'isosmiles':[' '.join(i) for i in example['isosmiles']]}
# dataset=dataset.map(addsplit,remove_columns=dataset.column_names,batched=True)
dataset['canonicalsmiles'][0]

'CC(=O)OC(CC(=O)O)C[N+](C)(C)C'

In [2]:
dataset['canonicalsmiles'][20]

'CCC(=O)C(=O)O'

In [3]:
def get_training_corpus():
    for i in range(0, len(dataset), 1000):
        yield dataset[i : i + 1000]["canonicalsmiles"]


In [4]:
from tokenizers import Tokenizer,models,pre_tokenizers,trainers,decoders,processors
tokenizer = Tokenizer(models.BPE())
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
tokenizer.pre_tokenizer.pre_tokenize_str("CC CC CC")


[('CC', (0, 2)), ('ĠCC', (2, 5)), ('ĠCC', (5, 8))]

In [5]:
trainer = trainers.BpeTrainer(vocab_size=120,special_tokens=['S(=O)','S(=O)(=O)','S(=S)','CS','CO','=O','(=O)','C=C','CC','OH','CHO','C=O','C=O(O)','COOH','C=N','NHC(O)','NO','NCN','NN','SS','(CN)','Cl','Br'])
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

In [8]:
encoding = tokenizer.encode("CC(C)CCCC")
print(encoding.tokens)
print(encoding.ids)
tokenizer.pre_tokenizer.pre_tokenize_str("CC(C)CCCC")


['CC', '(', 'C', ')', 'CC', 'CC']
[8, 25, 43, 26, 8, 8]


[('CC', (0, 2)), ('(', (2, 3)), ('C', (3, 4)), (')', (4, 5)), ('CCCC', (5, 9))]

In [0]:
# tokenizer.decoder = decoders.ByteLevel()

In [9]:
tokenizer.decode(encoding.ids)

'( C )'

In [13]:
from transformers import PreTrainedTokenizerFast

# wrapped_tokenizer = PreTrainedTokenizerFast(
#     tokenizer_object=tokenizer,
#     bos_token="<|endoftext|>",
#     eos_token="<|endoftext|>",
# )
from transformers import GPT2TokenizerFast

wrapped_tokenizer = GPT2TokenizerFast(tokenizer_object=tokenizer)

In [14]:
wrapped_tokenizer.save_pretrained('tokenizer_gpt_special.json')

('tokenizer_gpt_special.json\\tokenizer_config.json',
 'tokenizer_gpt_special.json\\special_tokens_map.json',
 'tokenizer_gpt_special.json\\vocab.json',
 'tokenizer_gpt_special.json\\merges.txt',
 'tokenizer_gpt_special.json\\added_tokens.json',
 'tokenizer_gpt_special.json\\tokenizer.json')